In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Bidirectional,Input
from tensorflow.keras.layers import BatchNormalization, TimeDistributed,LSTM,Dropout,Reshape
from tensorflow.keras import Model
from tensorflow.keras import Model
import numpy as np 
import pandas as pd
import os
from data import preproc as pp
from matplotlib import pyplot as plt
from PIL import Image
import random
import cv2

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
data_dir = '/home/austin/Documents/Github/handwritingnotebook/data/'

In [4]:
data_csv = pd.read_csv(data_dir+'words_csv/2020-06-03 11:39:42.000901.csv')

In [5]:
BATCH_SIZE = 100
IMG_HEIGHT = data_csv['height'].max()
IMG_WIDTH = data_csv['width'].max()
DATASET_SIZE = data_csv.shape[0]
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.,()?!':; "

In [6]:
IMG_HEIGHT

464

In [7]:
IMG_WIDTH

783

https://www.tensorflow.org/tutorials/load_data/images

In [8]:
#dataset = tf.data.TextLineDataset(data_dir+'words_csv/2020-06-03 11:39:42.000901.csv').skip(1)

In [9]:
def convert_to_onehot(data):
    #Creates a dict, that maps to every char of alphabet an unique int based on position
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    encoded_data = []
    #Replaces every char in data with the mapped int
    encoded_data.append([char_to_int[char] for char in data])
    encoded_data = encoded_data[0] # Prints the int encoded array

    #This part now replaces the int by an one-hot array with size alphabet
    letter = [0. for _ in range(len(alphabet))]
    for value in encoded_data:
        #At first, the whole array is initialized with 0
        #Only at the number of the int, 1 is written
        letter[value] = 1.
    letter = tf.convert_to_tensor(letter)
    return letter


In [10]:
# def preprocess_image(img_path,data_dir):
#     img_path = data_dir + 'words_screenshot_labeled/' + img_path
#     img = tf.image.decode_png(tf.io.read_file(img_path),channels = 3)
#     img = tf.image.rgb_to_grayscale(img)
#     img = tf.image.resize_with_pad(img,IMG_HEIGHT,IMG_WIDTH)
#     img = tf.image.transpose(img)
#     img = img/255
#     return img

In [11]:
def preprocess_image(img_path, data_dir, imgSize):
    img_path = data_dir + 'words_screenshot_labeled/' + img_path
    img = cv2.imread(img_path,0)
    (wt, ht) = imgSize
    (h, w) = img.shape
    fx = w / wt
    fy = h / ht
    f = max(fx, fy)
    newSize = (max(min(wt, int(w / f)), 1), max(min(ht, int(h / f)), 1)) # scale according to f (result at least 1 and at most wt or ht)
    img = cv2.resize(img, newSize)
    target = np.ones([ht, wt]) * 255
    target[0:newSize[1], 0:newSize[0]] = img

    # transpose for TF
    img = cv2.transpose(target)

    # normalize
    (m, s) = cv2.meanStdDev(img)
    m = m[0][0]
    s = s[0][0]
    img = img - m
    img = img / s if s>0 else img
    img = tf.convert_to_tensor(img)
    img = tf.expand_dims(img,2)
    print(img.get_shape())
    return img

In [12]:
def create_img_dataset(dataset,data_dir):
    img_dataset_list = []
    for index, row in dataset.iterrows():
        img_dataset_list.append(preprocess_image(row[0],data_dir,[128,32]))
    data_as_dataset = tf.data.Dataset.from_tensor_slices(img_dataset_list)
    return data_as_dataset

In [13]:
def create_label_dataset(dataset):
    label_dataset_list = []
    for index, row in dataset.iterrows():
        label_dataset_list.append(convert_to_onehot(row[1]))
    data_as_dataset = tf.data.Dataset.from_tensor_slices(label_dataset_list)
    return data_as_dataset

In [14]:
imgs = create_img_dataset(data_csv,data_dir)
labels = create_label_dataset(data_csv)

(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)
(128, 32, 1)

In [15]:
labeled_ds = tf.data.Dataset.zip((imgs,labels))

In [16]:
train_size = int(0.90 * DATASET_SIZE)
test_size = int(0.10 * DATASET_SIZE)

full_dataset = labeled_ds.shuffle(DATASET_SIZE)
train_ds = full_dataset.take(train_size)
train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=True)
test_ds = full_dataset.skip(train_size)
test_ds = test_ds.batch(BATCH_SIZE, drop_remainder=True)

In [17]:
test_ds

<BatchDataset shapes: ((100, 128, 32, 1), (100, 62)), types: (tf.float64, tf.float32)>

In [45]:
def build(input_shape, classes):
    # list of parameters for the layers
    kernelVals = [5, 3, 3, 3]
    featureVals = [64, 128, 128, 256]
    poolVals = [(2,2), (1,2), (1,2), (1,2)]
    numLayers = len(featureVals)
    
    model = models.Sequential()
    model.add(layers.Conv2D(filters = 32,
                                kernel_size = [5,5],
                                padding = 'SAME',
                                input_shape = (input_shape[0],input_shape[1],1)
                               )
                 )
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size = poolVals[0],strides = poolVals[0]))
    
    for i in range(numLayers):
        model.add(layers.Conv2D(filters = featureVals[i],
                                kernel_size = [kernelVals[i],kernelVals[i]],
                                padding = 'SAME'
                               )
                 )
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size = poolVals[i],strides = poolVals[i]))
    model.add(layers.Reshape((32,256)))
    model.add(Bidirectional(LSTM(256,return_sequences = True)))
    model.add(Bidirectional(LSTM(256,return_sequences = True)))
    model.add(layers.Reshape((32,1,512)))
    kernel = tf.Variable(tf.random.truncated_normal([1, 1, 512, 63], stddev=0.1))
    model.add(layers.Conv2D(filters = kernel, kernel_size = [1,1),strides = (1,1),padding = 'SAME'))
    return model

In [46]:
model = build([128,32,1],62)

In [47]:
OPTIMIZER = tf.keras.optimizers.Adam()

In [48]:
model.compile(loss = 'categorical_crossentropy',optimizer= OPTIMIZER,
             metrics=['accuracy'])

In [49]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 128, 32, 32)       832       
_________________________________________________________________
batch_normalization_40 (Batc (None, 128, 32, 32)       128       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 64, 16, 32)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 64, 16, 64)        51264     
_________________________________________________________________
batch_normalization_41 (Batc (None, 64, 16, 64)        256       
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 32, 8, 64)         0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 32, 8, 128)       